### \#6

Modify the codes found in the notes for the following Laplace equation using (m−2)(n−
2)-by-(m − 2)(n − 2) matrix problem by directly feeding the boundary condition to the solution: $\\$
$\begin{cases} \Delta u = 0 \\ u(x,0) = \text{sin}(\pi x) \\
u(x,1) = \text{sin}(\pi x) \\ u(0,y) = 0 \\ u(1,y) = 0 \end{cases}$

### Converting the indexing

In [1]:
import numpy as np
from internallib import tridiag
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16

def convert_ind(ind, shape):
    """
    Convert a 1D index to a 2D index, or vice versa.

    Input:
        ind: (int or tuple) Index to be converted. 
            2D index (i, j) is given, i corresponds to x location,
            and j corresponds to y location. 
        shape: (tuple) Shape of the 2D array.
    Note:

    CAUTION: 
        2nd dim, m, is the length of grid in x direction (horizontal)
        but, the 1st component of ind (e.g., i of (i, j)) gives the 
        the location of x. This is why the order of ind is opposite 
        depending on "1D to 2D" or "2D to 1D".
        Phylosophy: (i, j) means really mathematical grid index so that
            it corresponds to (x, y) coordinate while 1D index is 
            converted into array index, hence 
            (row, col) <--> (y, x) numpy array.
    """
    # Assume shape is (n, m) or n-by-m
    m = shape[1]

    if isinstance(ind, int) or len(ind) == 1:
        return (ind//m, ind%m)
    elif len(ind) == 2:
        return ind[1]*m + ind[0]
    else:
        raise ValueError("The input type is not supported.")

#### Test for index conversion

In [ ]:
n = 5
m = 7
shape = (n, m)

# test 1D to 2D
#   here second component of ind is the x location
#   so that the output can be used as ndarray index.
for i in range(n*m):
    print(i, convert_ind(i, shape))

#### Showing relation between original indices and coverted indices

In [ ]:
# CAUTION: i ranges from 0 to m-1, and j ranges from 0 to n-1
#       (i, j) for n-by-m
#   Here, the first component of ind is the x location
#   so that it matches the mathematical grid domain.
#   Index engineering is done behind the scene.

for j in range(n):
    for i in range(m):
        print((i, j), convert_ind((i, j), shape))

### Vector and Matrix Creation

In [4]:
def eval_2Dfun(f, x, y):
    """
    Return 2D array of evaluation f(x, y)

    Input:
        f: (function) Function to be evaluated.
        x: (1D array) x coordinate.
        y: (1D array) y coordinate.
    Output:
        f_vec: (2D array) Evaluation of f(x, y). Watch the flip of shape and index.
            Shape is (len(y), len(x)).
            f_vec[i, j] = f(x[j], y[i])
    """
    return f(x, y.reshape(-1, 1))

In [ ]:
n = 4
m = 7

fn = lambda x, y: x + y

x = np.arange(m)
y = np.arange(n)*10
f_grid = eval_2Dfun(fn, x, y)
print(f_grid)

#### Converting Array to Vector 

In [6]:
def to_vec(arr, shape=None):
    """
    Convert 2D array to 1D array in the context of 
    solving 2D elliptic problem using finite different method.

    Input:
        arr: (2D array) grid function.
        shape: (tuple) shape of the grid domain. Currently, not used.
    Output:
        vec: (1D array) 1D array of arr.
    """
    return arr.flatten()

#### Testing Conversion and Comparing Indexing Conventions

In [ ]:
# test `to_vec`
f_vec = to_vec(f_grid, (n, m))
print(f_vec)

# grid_fn[convert_ind(1D_index, shape)] = to_vec(grid_fn, shape)[1D_index]
print("\n", f"{'f_grid[2D index]' : ^20}{'f_vec[1D index]' : ^20}")
for i in range(n*m):
    print(f"{f_grid[convert_ind(i, shape)] : ^20}{to_vec(f_grid, shape)[i] : ^20}")

### Putting it Together 

In [8]:
def solve_pois_eq(f, bc_b, bc_r, bc_t, bc_l, xl, xr, yb, yt, n, m):
    """
    Return numerical solution of Poisson equation on a rectangular domain 
    using finite difference method.
    
    Input:
        f: (function) right hand side function.
        bc_b: (function) Boundary condition at bottom.
        bc_r: (function) Boundary condition at right.
        bc_t: (function) Boundary condition at top.
        bc_l: (function) Boundary condition at left.
        xl: (float) Left boundary of the domain.
        xr: (float) Right boundary of the domain.
        yb: (float) Bottom boundary of the domain.
        yt: (float) Top boundary of the domain.
        m: (int) Number of spatial grid in x direction.
        n: (int) Number of spatial grid in y direction.
    Output:
        w: (2D array) Numerical solution.
    """
    # Create discrete domain
    x = np.linspace(xl, xr, m)
    y = np.linspace(yb, yt, n)

    h = x[1] - x[0]
    k = y[1] - y[0]

    shape = (n, m)

    # Construct loading vector
    f_vec = eval_2Dfun(f, x, y)
    b = to_vec(f_vec) # convert 2D array to 1D array

    # Construct coefficient matrix
    A = np.zeros((n*m, n*m))
    for i in range(1, m-1):
        for j in range(1, n-1):
            p = convert_ind((i, j), shape)
            A[p, p] = -2./(h*h) - 2./(k*k)
            A[p, convert_ind((i+1, j), shape)] = 1./(h*h)
            A[p, convert_ind((i-1, j), shape)] = 1./(h*h)
            A[p, convert_ind((i, j+1), shape)] = 1./(k*k)
            A[p, convert_ind((i, j-1), shape)] = 1./(k*k)

    # Initialize solution vector
    w = np.zeros(n*m)

    # Apply boundary condition
    for j in [0, n-1]:
        for i in range(m):
            p = convert_ind((i, j), shape)
            A[p, p] = 1.
            b[p] = bc_b(x[i]) if j == 0 else bc_t(x[i])
    
    for i in [0, m-1]:
        for j in range(n):
            p = convert_ind((i, j), shape)
            A[p, p] = 1.
            b[p] = bc_l(y[j]) if i == 0 else bc_r(y[j])
    
    # Solve the linear system
    w = np.linalg.solve(A, b)
    w = w.reshape(n, m)

    return w, x, y

### Setting values for the problem

In [9]:
xl = 0.
xr = 1.
yb = 0.
yt = 1.

n = 11
m = 11

# Note: f = lambda x, y: 0. returns 0., which is float. 
#   This lacks methods of numpy arrar.
f = lambda x, y: 0.*x + 0.*y

bc_b = lambda x: np.sin(np.pi*x)
bc_r = lambda y: 0.
bc_t = lambda x: np.sin(np.pi*x)
bc_l = lambda y: 0.


w, x, y = solve_pois_eq(f, bc_b, bc_r, bc_t, bc_l, xl, xr, yb, yt, n, m)

### Plotting

In [ ]:
#%% plot
fig, ax = plt.subplots(1,1, figsize=(6.5, 6.5), subplot_kw={'projection':'3d'})

# surface plot (toggle)
# Note: need to reshape 1st coordinate to a column vector
ax.plot_surface(x, y.reshape(-1,1), w, cmap='viridis', alpha=0.8)
ax.set_title('Numerical solution\nLaplace equation')

# Change the angle of projection
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.view_init(elev=20, azim=60)

plt.show()